In [1]:
import os
import json
import hashlib
from dateutil import parser
from datetime import datetime

## cleanup erpnext

In [2]:
# cleanup projects
# cleanup task
# cleanup timesheet
# cleanup timelogs

In [2]:
from util.frappyclient import FrappeClient

In [3]:
%env ACERP_ADMINISTRATOR=Administrator
%env ACERP_ADMINISTRATOR_PASS=Pandion123456@

env: ACERP_ADMINISTRATOR=Administrator
env: ACERP_ADMINISTRATOR_PASS=Pandion123456@


In [4]:
conn = FrappeClient("https://acerp.pandion.vn")
conn.login(os.environ['ACERP_ADMINISTRATOR'], os.environ['ACERP_ADMINISTRATOR_PASS'])

{'message': 'Logged In', 'home_page': '/app', 'full_name': 'Administrator'}

In [5]:
# cleanup projects

In [126]:
conn.insert({
    "doctype": "Task",
    "subject": "subject-test",
    "type": "Other",
})

{'name': '7pjj0brhgr',
 'owner': 'Administrator',
 'creation': '2024-05-23 11:46:54.758959',
 'modified': '2024-05-23 11:46:54.758959',
 'modified_by': 'Administrator',
 'docstatus': 0,
 'idx': 0,
 'subject': 'subject-test',
 'type': 'Other',
 'is_group': 0,
 'is_template': 0,
 'status': 'Open',
 'priority': 'Low',
 'task_weight': 0.0,
 'expected_time': 0.0,
 'start': 0,
 'progress': 0.0,
 'duration': 0,
 'is_milestone': 0,
 'depends_on_tasks': '',
 'actual_time': 0.0,
 'total_costing_amount': 0.0,
 'total_expense_claim': 0.0,
 'total_billing_amount': 0.0,
 'company': 'ACONS',
 'lft': 579,
 'rgt': 580,
 'old_parent': '',
 'doctype': 'Task',
 'depends_on': []}

In [114]:
conn.get_doc('Task', 'i5f8vf2csr')

{'name': 'i5f8vf2csr',
 'owner': 'Administrator',
 'creation': '2024-05-22 16:05:39.991552',
 'modified': '2024-05-22 16:05:39.991552',
 'modified_by': 'Administrator',
 'docstatus': 0,
 'idx': 0,
 'subject': 'THIẾT KẾ',
 'project': 'P39924_VTD.00',
 'type': 'Other',
 'is_group': 1,
 'is_template': 0,
 'status': 'Open',
 'priority': 'Low',
 'task_weight': 0.0,
 'expected_time': 0.0,
 'start': 0,
 'progress': 0.0,
 'duration': 0,
 'is_milestone': 0,
 'depends_on_tasks': '',
 'actual_time': 0.0,
 'total_costing_amount': 0.0,
 'total_expense_claim': 0.0,
 'total_billing_amount': 0.0,
 'company': 'ACONS',
 'lft': 557,
 'rgt': 558,
 'old_parent': '',
 'doctype': 'Task',
 'depends_on': []}

In [5]:
doc = conn.get_doc('Timesheet', 'TS-22-05-24-05059_HR-EMP-00023')

In [6]:
doc

{'name': 'TS-22-05-24-05059_HR-EMP-00023',
 'owner': 'Administrator',
 'creation': '2024-05-22 19:18:48.890100',
 'modified': '2024-05-22 19:18:48.890100',
 'modified_by': 'Administrator',
 'docstatus': 0,
 'idx': 0,
 'title': 'Nhung Nguyễn Thị Hồng',
 'naming_series': 'TS-.YYYY.-',
 'company': 'ACONS',
 'currency': 'VND',
 'exchange_rate': 1.0,
 'status': 'Draft',
 'parent_project': 'HR01_ACO.00',
 'employee': 'HR-EMP-00023',
 'employee_name': 'Nhung Nguyễn Thị Hồng',
 'department': 'TEAMCIVIL-AC',
 'start_date': '2024-05-22',
 'end_date': '2024-05-22',
 'total_hours': 8.0,
 'total_billable_hours': 0.0,
 'base_total_billable_amount': 0.0,
 'base_total_billed_amount': 0.0,
 'base_total_costing_amount': 0.0,
 'total_billed_hours': 0.0,
 'total_billable_amount': 0.0,
 'total_billed_amount': 0.0,
 'total_costing_amount': 0.0,
 'per_billed': 0.0,
 'doctype': 'Timesheet',
 'time_logs': [{'name': 'lmkp734dec',
   'owner': 'Administrator',
   'creation': '2024-05-22 19:18:48.890100',
   'modi

In [7]:
doc = conn.get_doc('Timesheet', 'TS-04-04-24-00293_HR-EMP-00007')

In [144]:
# cleanup timesheets
ts = conn.get_list('Timesheet', 
                   fields = ['name', 'doctype', 'status', 'creation'],
                   filters = [['creation', '>=', '2024-01-01']], limit_page_length=1000)
# ts = conn.get_list('Timesheet', fields = ['name', 'creation'])
# ts = conn.get_list('Timesheet', filters = {'creation': {'=': '2024-01-31 12:10:29.497799'}})

In [145]:
len(ts)

0

In [146]:
ts[0:10]

[]

In [143]:
# delete NO RUN
for t in ts:
    if t['status'] == 'Draft' or t['status'] == 'Cancelled':
        conn.delete('Timesheet', t['name'])
    elif t['status'] == 'Submitted':
        t['doctype'] = 'Timesheet'
        t['status'] = 'Cancelled'
        t['docstatus'] = 2
        conn.update(t)
        # conn.set_value('Timesheet', t['name'], 'status', 'Cancelled')
        conn.delete('Timesheet', t['name'])

In [ ]:
# cleanup tasks

In [21]:
tsk = conn.get_list('Task', 
                    fields = ['name', 'status', 'project', 'creation', 'is_group'],
                    filters = [['creation', '>=', '2024-01-01'], ['is_group', '=', '1']], limit_page_length=300)

In [22]:
len(tsk)

0

In [19]:
# tsk[0:10]

In [20]:
# delete NO RUN
for t in tsk:
    print(t['name'])
    t['doctype'] = 'Task'
    t['depends_on'] = []
    conn.update(t)
    tsk_child = conn.get_list('Task', 
                              fields = ['name', 'status', 'project', 'creation', 'is_group'],
                              filters = [['parent_task', '=', t['name']]], limit_page_length=1000)
    for tt in tsk_child:
        tt['doctype'] = 'Task'
        tt['parent_task'] = ''
        conn.update(tt)
        conn.delete('Task', tt['name'])
    # print(t['name'])
    conn.delete('Task', t['name'])

eebe8b4cf6
eedff68d84
ef8b5cc518
efa1419452
efa8742421
f0322607b4
f0948c7183
f120cff99d
f1b82a61f7
f23abb0c82
f377b82dfb
f3a5bb1db3
f3b431416b
f42362308c
f433d06c3e
f49760f142
f539f1feb7
f57fc1079d
f5a79b376a
f5cdd275ef
f756508623
f75d90bcda
f7bb222db3
f7d0f3532e
f871fb7f2b
fa184b6f8e
fa4f42459a
fa577361f7
faf91d8c15
fb2d1c48e7
fb6ef2782d
fbd6c4a9d8
fc22670c46
fccfc74c7b
fd925479b0
fe3d3e1cac
fedcde79bc
ff04316de8
ff5e940552
ffd2f3ad8d


In [23]:
tsk = conn.get_list('Task', 
                    fields = ['name', 'status', 'project', 'creation', 'is_group'],
                    filters = [['creation', '>=', '2024-01-01']], limit_page_length=300)

In [24]:
len(tsk)

34

In [25]:
# delete NO RUN
for tt in tsk:
    tt['doctype'] = 'Task'
    tt['parent_task'] = ''
    conn.update(tt)
    conn.delete('Task', tt['name'])

###

In [217]:
# tsk[1]

In [220]:
# for t in tsk:
#     conn.delete('Task', t['name'])

In [218]:
# conn.delete('Task', 'e38sn3pg90')

In [205]:
# for t in tsk:
#     print(t['name'])
#     conn.delete('Task', t['name'])

In [188]:
# tsk[0]['doctype'] = 'Task'
# tsk[0]['depends_on'] = []
# conn.update(tsk[0])

In [206]:
# conn.get_doc('Task', 'e38p1tg5gg')

In [190]:
# conn.delete('Task', 'e38p1tg5gg')

In [165]:
# conn.get_doc('Task', 'lia3u8u03v')

In [161]:
# for t in tsk:
#     conn.delete('Task', t['name'])

## timesheet import with google sheet

#### process flow

In [1]:
# handle sheet file
    # for each row of sheets
        # handle row data
            # if row is empty
                # skip this row
            # if col_B == "P"
                # handle as parent task
                # if col_A is not empty (parent task is created)
                    # handle update parent task if any changes (compare hash)
                # else
                    # handle as new parent task
            # else
                # handle as task with timesheet
                # if col_A is not empty (timesheet is created)
                    # handle update timesheet if any changes (compare hash)
                # else
                    # handle as new timesheet
    # endfor
    # handle update col_A data of sheets
        # for each row of col_A array above
            # update batch of col_A in sheet
    # end of function

#### problems

In [ ]:
# 1. col_A: can be duplicate (user copy and paste with new row)
# 2. col_A: 
# 3.
# 4.

In [5]:
from util.google_sheet import GGSheet

In [6]:
EXCEL_TASK_PRIORITY = {
    "": "Medium",
    "1_Urgen": "Urgent",
    "2_Important": "High",
    "3_Medium": "Medium",
    "4_Low": "Low",
}
EXCEL_TYPE_PARENT_TASK = {
    "": "Other",
    "0_Pre CO": "Pre Concept",
    "1_CO": "Concept",
    "2_BD": "Basic Design",
    "3_DD": "Design Development",
    "4_TD": "Tender Doc",
    "5_CD": "Construction",
    "6_AU": "Authorship",
    "7_Other": "Other",
}
EXCEL_TASK_STATUS = {
    "": "Open",
    "1_Open": "Open",
    "2_In progress": "In Progress",
    "3_Pending": "Pending",
    "4_Cancel": "Cancel",
    "5_Done": "Done",
    "6_Review": "Review",
}
EXCEL_TIME_SHEET_STATUS = {
    "": "Draft",
    "Open": "Draft",
    "In Progress": "Draft",
    "Pending": "Draft",
    "Review": "Draft",
    "Done": "Submitted",
    "Cancel": "Cancelled",
}
EXCEL_TIME_SHEET_DOC_STATUS = {
    "": 0,
    "Draft": 0,
    "Submitted": 1,
    "Cancelled": 2,
}

In [7]:
row_of_date = 3
row_start   = 5
row_end     = 1200
# company     = "ACONS"

In [8]:
# url_file="https://docs.google.com/spreadsheets/d/12bDZbCnJVkFTJTeMBmgsrAEejVKQCXa11M951ylOQVw/edit#gid=1885142845"
# worksheet_name="BAP"

In [9]:
# url_file = "https://docs.google.com/spreadsheets/d/1rgen0jYUoAzHRelVi6c2ATJs3IzYPD20wMcT63inKoI/edit#gid=996503932"
# worksheet_name="T2"

In [10]:
# ggSheet = GGSheet(url_file, worksheet_name)

In [11]:
# def mapping_row_with_attr(row_data):
#     attrs = {}
#     # no need anymore
#     attrs["import_key"]          = row_data["A"] if "A" in row_data else ""
#     # import_key based on
#     # task
#     # new_key = f"{payload.expected_start_date};{payload.expected_end_date};{payload.new_end_date}"
#     # new_hash_key = hash_str_8_dig(new_key)
#     # prev_hash_key, _, __ = split_str_get_key(input_data=payload.prev_hash_key, char_split = "--")
#     # timesheet
#     # new_key = f"{project_code};{parent_task};{employee_name};{progress};{activity_code};{task};{excel_task_status};{date_string}"
#     # new_hash_key = hash_str_8_dig(new_key)
#     # A_column_key = f"{new_hash_key}--{task_doc}--{new_time_sheet_doc.name}"
#     # prev_hash_key, task_id, time_sheet_id = split_str_get_key(input_data=cell["A"], char_split="--")
    
#     attrs["row_type"]            = row_data["B"] if "B" in row_data else ""
#     attrs["project_code"]        = row_data["C"] if "C" in row_data else ""
#     attrs["project_name"]        = row_data["D"] if "D" in row_data else ""

#     attrs["expected_start_date"] = parser.parse(row_data["E"]) if "E" in row_data and row_data["E"] != "" else None
#     attrs["expected_end_date"]   = parser.parse(row_data["F"]) if "F" in row_data and row_data["F"] != "" else None
#     attrs["new_end_date"]        = parser.parse(row_data["G"]) if "G" in row_data and row_data["G"] != "" else None
#     attrs["phase_name"]          = EXCEL_TYPE_PARENT_TASK[row_data["H"] if "H" in row_data else ""]
#     # attrs["phase_name"           = EXCEL_TYPE_PARENT_TASK[row_data["H"] if "H" in row_data else ""]
#     # expected_time = float(cell["I"]) if "I" in cell and cell["I"] != "" else 0.0
#     attrs["expected_time"]       = float(row_data["I"]) if "I" in row_data and row_data["I"] != "" else 0.0
#     # col_J: real time (sum)
#     attrs["task_priority"]       = EXCEL_TASK_PRIORITY[row_data["K"] if "K" in row_data else ""]
#     attrs["progress"]            = row_data["L"].replace("%", "") if "L" in row_data else ""
#     attrs["employee_name"]       = row_data["M"] if "M" in row_data else ""
#     attrs["activity_code"]       = row_data["N"] if "N" in row_data else ""
#     if attrs["activity_code"] == "":
#         attrs["activity_code"] = "0000"
#     attrs["task"]                = row_data["O"] if "O" in row_data else ""
#     attrs["subject"]             = row_data["O"] if "O" in row_data else ""
#     if attrs["subject"] == "":
#         attrs["subject"] = "Other"
#     attrs["excel_task_status"]   = EXCEL_TASK_STATUS[row_data["P"] if "P" in row_data else ""]
#     attrs["task_status"]         = EXCEL_TASK_STATUS[row_data["P"] if "P" in row_data else ""]
#     # self.task_number = num
    
#     return attrs

In [12]:
# def mapping_row_with_time_log(row_data, row_date_header):
#     new_date = {}
#     date_string = ""
#     for column, value in row_data.items():
#         if column in row_date_header and value != None and value != "":
#             date = parser.parse(row_date_header[column])
#             new_date[date] = value
#             date_string = date_string + column + "-" + value + ";"
#     return new_date, date_string

In [13]:
# def hash_str_8_dig(raw_str):
#     encode = hashlib.sha1(raw_str.encode("utf-8")).hexdigest()
#     hash_obj = int(encode, 16) % (10 ** 8)
#     return str(hash_obj)

In [14]:
# cheap_hash = lambda input: hashlib.md5(input.encode("utf-8")).hexdigest()[:8]

In [15]:
# cheap_hash("pandion.vn")

In [16]:
# for caching
# project_doc_list = {}
# task_doc_list = {}
# employee_doc_list = {}
# timesheet_doc_list = {}

# for update sheet
# update_records = {}

In [17]:
# def is_row_empty(row_attrs):
#     if row_attrs['project_code'] == "":
#         return True
#     return False

In [18]:
# def is_row_timesheet_empty(row_attrs):
#     if row_attrs['project_code'] == "":
#         return True
#     return False

In [19]:
# def is_time_log_row_empty(row_time_logs):
#     return True

In [20]:
# def create_task():
#     pass

In [21]:
# def update_task():
#     pass

In [22]:
# def create_timesheet():
#     pass

In [23]:
# def update_timesheet():
#     pass

In [24]:
# def get_project_doc(doc_name):
#     # return if cached else get db & cache
#     if project_doc_list.get(doc_name):
#         return project_doc_list[doc_name]
#     else:
#         doc = conn.get_doc('Project', doc_name)
#         if doc:
#             project_doc_list[doc_name] = doc
#             return doc

#     return None

In [25]:
# def get_employee_doc(doc_name):
#     # return if cached else get db & cache
#     if employee_doc_list.get(doc_name):
#         return employee_doc_list[doc_name]
#     else:
#         doc = conn.get_doc('Employee', doc_name)
#         if doc:
#             employee_doc_list[doc_name] = doc
#             return doc
#     return None

In [26]:
# def get_employee_doc_by_name(employee_name):
    
#     doc_name = conn.get_value("Employee", 
#                               ["name"],
#                               [["employee_name", "=", employee_name]])
#     print(f"employee: {doc_name}")
#     if doc_name:
#         return get_employee_doc(doc_name['name'])
        
#     return None

In [27]:
# def get_task(doc_name):
#     # return if cached else get db & cache
#     if task_doc_list.get(doc_name):
#         return task_doc_list[doc_name]
#     else:
#         doc = conn.get_doc('Task', doc_name)
#         if doc:
#             task_doc_list[doc_name] = doc
#             return doc
#     return None

In [47]:
# def get_task_doc_by_fields(subject, project_code, phase_name, is_group=0, parent_task=""):

#     filters = [["subject", "=", subject],
#                ["project", "=", project_code], 
#                ["type", "=", phase_name],
#                ["is_group", "=", is_group]]
#     if parent_task != "":
#         filters.append(["parent_task", "=", parent_task])
    
#     doc_name = conn.get_value("Task", ["name"], filters)
#     if doc_name:
#         # print(f"doc_name existed")
#         return get_task(doc_name["name"])
#     return None

In [29]:
# def get_timesheet(doc_name):
#     # return if cached else get db & cache
#     if timesheet_doc_list.get(doc_name):
#         return timesheet_doc_list[doc_name]
#     else:
#         doc = conn.get_doc('Timesheet', doc_name)
#         if doc:
#             timesheet_doc_list[doc_name] = doc
#             return doc
#     return None
#     pass

In [30]:
# def create_task(payload):
#     return conn.insert(payload)
#     # pass

In [31]:
# def update_task(payload):
#     pass

In [32]:
# def create_timesheet(payload):
#     conn.insert(payload)

In [33]:
# def update_timesheet(name, payload):
#     pass

In [34]:
# import_key for task: task_id||
def handle_timesheet_row_with_parent_task(idx, row_data, row_attrs, row_date_header, company):

    # print(f"  Handle parent task at row: {idx}\n"
    #       f"  import_key: {row_attrs['import_key']}\n"
    #       f"  project_code: {row_attrs['project_code']}\n"
    #       f"  phase_name: {row_attrs['phase_name']}\n"
    #       f"  subject: {row_attrs['subject']}\n"
    #       f"  task: {row_attrs['task']}")
    task_doc = get_task_doc_by_fields(row_attrs['subject'], row_attrs['project_code'], row_attrs['phase_name'], 1)

    if task_doc:
        return task_doc
    else:
        # create new
        payload = {}
        payload["doctype"] = "Task"
        payload["subject"] = row_attrs["subject"]
        payload["project"] = row_attrs["project_code"]
        payload["type"] = row_attrs["phase_name"]
        payload["is_group"] = 1
        print(f"{payload}")
        return create_task(payload)

In [49]:
# import_key for task: timesheet_id||
def handle_timesheet_row_as_task(idx, row_data, row_attrs, row_date_header, 
                                 parent_task_row, parent_task_doc, company):
    time_logs_date, time_logsdate_string = mapping_row_with_time_log(row_data, row_date_header)
    
    # print(f"  Handle task at row: {idx}\n"
    #       f"  import_key: {row_attrs['import_key']}\n"
    #       f"  project_code: {row_attrs['project_code']}\n"
    #       f"  phase_name: {row_attrs['phase_name']}\n"
    #       # f"  subject: {row_attrs['subject']}\n"
    #       f"  employee_name: {row_attrs['employee_name']}\n"
    #       f"  task: {row_attrs['task']}\n"
    #       f"  parent task row: {parent_task_row}\n"
    #       f"  parent task doc: {parent_task_doc}\n"
    #       f"  time_logs: {time_logs_date}")

    doc_name = conn.get_value("Timesheet", 
                              ["name"],
                              [["employee", "=", row_attrs['employee_code']],
                               ["parent_project", "=", row_attrs['project_code']],
                               ["custom_task", "=", row_attrs['task']],
                               ["custom_phase", "=", row_attrs['phase_name']],])

    if doc_name:
        print(f"Timesheet doc_name existed {doc_name}")
        
        # existed
        payload = {}
        payload["doctype"] = "Timesheet"
        payload["name"] = doc_name["name"]
        payload["employee"] = row_attrs["employee_code"]
        payload["customer"] = company
        payload["parent_project"] = row_attrs["project_code"]
        payload["custom_task"] = row_attrs["task"]
        payload["custom_phase"] = row_attrs["phase_name"]
        # payload["start_date"] = row_attrs["phase_name"]
        time_logs = []
        for date, hrs in time_logs_date.items():
            # check task here for update or new
            task_doc = get_task_doc_by_fields(row_attrs['subject'], 
                                              row_attrs['project_code'], 
                                              row_attrs['phase_name'], 0, parent_task_doc['name'])
            # task_doc_name = conn.get_value("Task", 
            #                               ["name"],
            #                               [["subject", "=", row_attrs['subject']],
            #                                ["project", "=", row_attrs['project_code']], 
            #                                ["type", "=", row_attrs['phase_name']],
            #                                ["is_group", "=", 0], 
            #                                ["parent_task", "=", parent_task_doc['task_doc']['name']]])
            # if task_doc_name:
            #     # print("task is existed")
            #     task_doc = get_task(task_doc_name['name'])
            # else:
            if task_doc is None:
                print("task is not exists")
                # new task for timelogs
                task_payload = {}
                task_payload["doctype"] = "Task"
                task_payload["subject"] = row_attrs["subject"]
                task_payload["project"] = row_attrs["project_code"]
                task_payload["type"] = row_attrs["phase_name"]
                task_payload["is_group"] = 0
                task_payload["parent_task"] = parent_task_doc['task_doc']['name']
                # print(f"{task_payload}")
                task_doc = create_task(task_payload)
                # print(f"new task doc: {task_doc}")

            if task_doc is None:
                print("fixme")
                continue
                
            time_logs.append({
                    "activity_type": row_attrs['activity_code'],
                    "from_time": date.strftime("%Y-%m-%d"),
                    "hours": float(hrs),
                    "project": row_attrs['project_code'],
                    "task": task_doc['name'],
                    # "completed": excel_task_status == "Done",
                })
        payload["time_logs"] = time_logs
        # payload["is_group"] = 1
        print(f"{payload}")
        conn.update(payload)
        # pass
    else:
        # create new
        payload = {}
        payload["doctype"] = "Timesheet"
        payload["employee"] = row_attrs["employee_code"]
        payload["parent_project"] = row_attrs["project_code"]
        payload["customer"] = company
        payload["custom_task"] = row_attrs["task"]
        payload["custom_phase"] = row_attrs["phase_name"]
        # payload["start_date"] = row_attrs["phase_name"]

        time_logs = []
        for date, hrs in time_logs_date.items():
            # check task here for update or new
            task_doc = get_task_doc_by_fields(row_attrs['subject'], 
                                              row_attrs['project_code'], 
                                              row_attrs['phase_name'], 0, parent_task_doc['name'])
            if task_doc is None:
                print("task is not exists")
                # new task for timelogs
                task_payload = {}
                task_payload["doctype"] = "Task"
                task_payload["subject"] = row_attrs["subject"]
                task_payload["project"] = row_attrs["project_code"]
                task_payload["type"] = row_attrs["phase_name"]
                task_payload["is_group"] = 0
                task_payload["parent_task"] = parent_task_doc['task_doc']['name']
                # print(f"{task_payload}")
                task_doc = create_task(task_payload)
                # print(f"new task doc: {task_doc}")

            if task_doc is None:
                print("fixme")
                continue
            
            time_logs.append({
                    "activity_type": row_attrs['activity_code'],
                    "from_time": date.strftime("%Y-%m-%d"),
                    "hours": float(hrs),
                    "project": row_attrs['project_code'],
                    "task": task_doc['name'],
                    # "completed": excel_task_status == "Done",
                })
        payload["time_logs"] = time_logs
        # payload["is_group"] = 1
        print(f"{payload}")
        create_timesheet(payload)

    return True

In [36]:
current_parent_task_row = 0
current_parent_task_doc = {}

In [37]:
# if row is empty
    # skip this row
# if col_B == "P"
    # handle as parent task
    # if col_A is not empty (parent task is created)
        # handle update parent task if any changes (compare hash)
    # else
        # handle as new parent task
# else
    # handle as task with timesheet
    # if col_A is not empty (timesheet is created)
        # handle update timesheet if any changes (compare hash)
    # else
        # handle as new timesheet
def handle_timesheet_row(idx, row_data, row_date_header, company):
    global current_parent_task_row
    global current_parent_task_doc
    row_attrs = mapping_row_with_attr(row_data)
    # NO project code
    if is_row_empty(row_attrs):
        return False

    # project code is not found ?
    print(f"Project code: {row_attrs['project_code']}")
    project_doc = get_project(row_attrs['project_code'])
    if project_doc is None:
        print(f"Project is not found: {row_attrs['project_code']}")
        return False
    
    if row_attrs['row_type'] == "P":
        # handle as parent task
        # if row_attrs['project_code'] in ["HR01_ACO.00"]:
        #     return
        ret = handle_timesheet_row_with_parent_task(idx, row_data, row_attrs, row_date_header, company)
        current_parent_task_row = idx
        current_parent_task_doc = ret
        # {
        #     # 'row_data': row_data,
        #     # 'row_attrs': row_attrs,
        #     'task_doc': ret
        # }
        return True
    else:
        # employee name is not found?
        print(f"Employee name: {row_attrs['employee_name']}")
        employee_doc = get_employee_doc_by_name(row_attrs['employee_name'])
        if employee_doc is None:
            print(f"Employee is not found: {row_attrs['employee_name']}")
            return False
        else:
            print(f"Employee code: {employee_doc['name']}")
            row_attrs['employee_code'] = employee_doc['name']
        # return False
        # handle as task with timesheet
        ret = handle_timesheet_row_as_task(idx, row_data, row_attrs, row_date_header, 
                                            current_parent_task_row, current_parent_task_doc, company)
        return ret
        

In [38]:
def handle_timesheet_sheet():
    try:
        # row_date_header = await get_row_date_header()
        # all_records = await ggSheet.get_all_row_values()
        
        current_parent_task_row = 0
        for idx, row_data in enumerate(all_records):
            if idx >= row_start and idx <= row_end:
                print(f"idx: {idx}")
                # print(row_data)
                handle_timesheet_row(idx, row_data, row_date_header, company)
                # break
    except Exception as err:
        print(f"Handle timesheet {worksheet_name} failed with: {err}")

In [39]:
url_file="https://docs.google.com/spreadsheets/d/12bDZbCnJVkFTJTeMBmgsrAEejVKQCXa11M951ylOQVw/edit#gid=1885142845"
worksheet_name="BAP"

In [40]:
ggSheet = GGSheet(url_file, worksheet_name)

In [41]:
row_date_header = await ggSheet.get_row_date_header(row_of_date)

In [42]:
all_records = await ggSheet.get_all_row_values()

In [43]:
len(employee_doc_list)

0

In [44]:
len(task_doc_list)

0

In [45]:
len(project_doc_list)

0

## BAP Team

In [50]:
company     = "BAP"
handle_timesheet_sheet()

idx: 5
Project code: HR01_ACO.00
idx: 6
Project code: HR01_ACO.00
Employee name: Nhật Phạm Hoàng
employee: {'name': 'HR-EMP-00007'}
Employee code: HR-EMP-00007
Timesheet doc_name existed {'name': 'TS-23-05-24-05061_HR-EMP-00007'}
{'doctype': 'Timesheet', 'name': 'TS-23-05-24-05061_HR-EMP-00007', 'employee': 'HR-EMP-00007', 'customer': 'BAP', 'parent_project': 'HR01_ACO.00', 'custom_task': 'Nghỉ lễ', 'custom_phase': 'Other', 'time_logs': [{'activity_type': '2000', 'from_time': '2024-04-27', 'hours': 8.0, 'project': 'HR01_ACO.00', 'task': '7iks43j1g1'}, {'activity_type': '2000', 'from_time': '2024-04-29', 'hours': 8.0, 'project': 'HR01_ACO.00', 'task': '7iks43j1g1'}, {'activity_type': '2000', 'from_time': '2024-04-30', 'hours': 8.0, 'project': 'HR01_ACO.00', 'task': '7iks43j1g1'}, {'activity_type': '2000', 'from_time': '2024-05-01', 'hours': 8.0, 'project': 'HR01_ACO.00', 'task': '7iks43j1g1'}]}
idx: 7
Project code: HR01_ACO.00
Employee name: Huyền Bùi Thị Thanh
employee: {'name': 'HR-EM

## TEAM_2

In [51]:
url_file = "https://docs.google.com/spreadsheets/d/1rgen0jYUoAzHRelVi6c2ATJs3IzYPD20wMcT63inKoI/edit#gid=996503932"
worksheet_name="T2"
company="ACO"

In [52]:
await ggSheet.set_worksheet(url_file, worksheet_name)

In [53]:
row_date_header = await ggSheet.get_row_date_header(row_of_date)
all_records = await ggSheet.get_all_row_values()

In [54]:
handle_timesheet_sheet()

idx: 5
Project code: P33422_HTP.00
idx: 6
Project code: P33422_HTP.00
Employee name: Bình Trịnh Thanh
employee: {'name': 'HR-EMP-00020'}
Employee code: HR-EMP-00020
Timesheet doc_name existed {'name': 'TS-23-05-24-05226_HR-EMP-00020'}
{'doctype': 'Timesheet', 'name': 'TS-23-05-24-05226_HR-EMP-00020', 'employee': 'HR-EMP-00020', 'customer': 'ACO', 'parent_project': 'P33422_HTP.00', 'custom_task': 'Thuyết minh TKTC', 'custom_phase': 'Construction', 'time_logs': []}
idx: 7
Project code: P33422_HTP.00
Employee name: Huy Phan Trịnh Hoàng
employee: {'name': 'HR-EMP-00069'}
Employee code: HR-EMP-00069
Timesheet doc_name existed {'name': 'TS-23-05-24-05227_HR-EMP-00069'}
{'doctype': 'Timesheet', 'name': 'TS-23-05-24-05227_HR-EMP-00069', 'employee': 'HR-EMP-00069', 'customer': 'ACO', 'parent_project': 'P33422_HTP.00', 'custom_task': 'Thuyết minh TKTC', 'custom_phase': 'Construction', 'time_logs': []}
idx: 8
Project code: P33422_HTP.00
Employee name: Bình Trịnh Thanh
employee: {'name': 'HR-EMP-0

## TEAM3

In [55]:
# team_3
# url_file="https://docs.google.com/spreadsheets/d/1hO5er83L8qU6UITbsjacZg_A6SPW8MuFRBEfwQd1dys/edit#gid=370020233"
# copied
url_file = "https://docs.google.com/spreadsheets/d/1LLKxyhIAbjMl8MXREZBGyH7r8puEtQATc5AiQLOIbh8/edit#gid=370020233"
worksheet_name="T3"
company="ACO"

In [56]:
await ggSheet.set_worksheet(url_file, worksheet_name)

In [57]:
row_date_header = await ggSheet.get_row_date_header(row_of_date)
all_records = await ggSheet.get_all_row_values()

In [58]:
handle_timesheet_sheet()

idx: 5
Project code: P000_ACO.00
idx: 6
Project code: P000_ACO.00
Employee name: Trung Đỗ Thành
employee: {'name': 'HR-EMP-00041'}
Employee code: HR-EMP-00041
Timesheet doc_name existed {'name': 'TS-23-05-24-05592_HR-EMP-00041'}
{'doctype': 'Timesheet', 'name': 'TS-23-05-24-05592_HR-EMP-00041', 'employee': 'HR-EMP-00041', 'customer': 'ACO', 'parent_project': 'P000_ACO.00', 'custom_task': 'Họp nội bộ', 'custom_phase': 'Other', 'time_logs': [{'activity_type': '2000', 'from_time': '2024-05-06', 'hours': 2.0, 'project': 'P000_ACO.00', 'task': 'd2946kmc0l'}]}
idx: 7
idx: 8
idx: 9
idx: 10
idx: 11
idx: 12
idx: 13
idx: 14
idx: 15
Project code: HR01_ACO.00
idx: 16
Project code: HR01_ACO.00
Employee name: Trung Đỗ Thành
employee: {'name': 'HR-EMP-00041'}
Employee code: HR-EMP-00041
Timesheet doc_name existed {'name': 'TS-23-05-24-05593_HR-EMP-00041'}
{'doctype': 'Timesheet', 'name': 'TS-23-05-24-05593_HR-EMP-00041', 'employee': 'HR-EMP-00041', 'customer': 'ACO', 'parent_project': 'HR01_ACO.00', 

## TEAM 4

In [59]:
# team_4
# url_file="https://docs.google.com/spreadsheets/d/1p1vffmF2H-rOSdrNjdqMAXYTFlf9HkBX7FJjk2l4OwA/edit#gid=1885142845"
# copied
url_file="https://docs.google.com/spreadsheets/d/11TQi-pF13BGS2LkrOHzO1EK-zAiZPw_Vop61d69oMFo/edit#gid=1885142845"
worksheet_name="T4"
company="ACO"

In [60]:
await ggSheet.set_worksheet(url_file, worksheet_name)

In [61]:
row_date_header = await ggSheet.get_row_date_header(row_of_date)
all_records = await ggSheet.get_all_row_values()

In [62]:
handle_timesheet_sheet()

idx: 5
Project code: HR01_ACO.00
idx: 6
Project code: HR01_ACO.00
Employee name: An Nguyễn Bình
employee: {'name': 'HR-EMP-00003'}
Employee code: HR-EMP-00003
Timesheet doc_name existed {'name': 'TS-23-05-24-05801_HR-EMP-00003'}
{'doctype': 'Timesheet', 'name': 'TS-23-05-24-05801_HR-EMP-00003', 'employee': 'HR-EMP-00003', 'customer': 'ACO', 'parent_project': 'HR01_ACO.00', 'custom_task': 'Nghỉ lễ giỗ tổ', 'custom_phase': 'Construction', 'time_logs': [{'activity_type': '4001', 'from_time': '2024-04-18', 'hours': 8.0, 'project': 'HR01_ACO.00', 'task': 'd62mj6kmal'}]}
idx: 7
Project code: HR01_ACO.00
Employee name: Son Trần Thị Minh
employee: {'name': 'HR-EMP-00005'}
Employee code: HR-EMP-00005
Timesheet doc_name existed {'name': 'TS-23-05-24-05802_HR-EMP-00005'}
{'doctype': 'Timesheet', 'name': 'TS-23-05-24-05802_HR-EMP-00005', 'employee': 'HR-EMP-00005', 'customer': 'ACO', 'parent_project': 'HR01_ACO.00', 'custom_task': 'Nghỉ lễ giỗ tổ', 'custom_phase': 'Construction', 'time_logs': [{'a

## TEAM 5

In [63]:
# team_5
# url_file="https://docs.google.com/spreadsheets/d/1vTy7Qout9WoeZa31lHLQUw6CZecjEJOKUQlDIAmk1vk/edit#gid=1742134070"
# copied
url_file="https://docs.google.com/spreadsheets/d/125OUpshHBj577ppiOUO_n73ZtgKDBBqLpmR0qST-IZA/edit#gid=1742134070"
worksheet_name="T5"
company="ACO"

In [64]:
await ggSheet.set_worksheet(url_file, worksheet_name)

In [65]:
row_date_header = await ggSheet.get_row_date_header(row_of_date)
all_records = await ggSheet.get_all_row_values()

In [66]:
handle_timesheet_sheet()

idx: 5
Project code: P39423_KNCR.00
idx: 6
Project code: P39423_KNCR.00
Employee name: 
employee: {}
Employee is not found: 
idx: 7
Project code: P39423_KNCR.00
Employee name: Thành Hồ Đức Thiện
employee: {'name': 'HR-EMP-00008'}
Employee code: HR-EMP-00008
Timesheet doc_name existed {'name': 'TS-23-05-24-06036_HR-EMP-00008'}
{'doctype': 'Timesheet', 'name': 'TS-23-05-24-06036_HR-EMP-00008', 'employee': 'HR-EMP-00008', 'customer': 'ACO', 'parent_project': 'P39423_KNCR.00', 'custom_task': 'Cập nhật phản hồi TKCS', 'custom_phase': 'Other', 'time_logs': [{'activity_type': '2303', 'from_time': '2024-04-01', 'hours': 8.0, 'project': 'P39423_KNCR.00', 'task': 'dco8m4aj1i'}, {'activity_type': '2303', 'from_time': '2024-04-02', 'hours': 8.0, 'project': 'P39423_KNCR.00', 'task': 'dco8m4aj1i'}, {'activity_type': '2303', 'from_time': '2024-04-03', 'hours': 8.0, 'project': 'P39423_KNCR.00', 'task': 'dco8m4aj1i'}, {'activity_type': '2303', 'from_time': '2024-04-04', 'hours': 8.0, 'project': 'P39423

## TEAM 6

In [67]:
# team_6
# url_file="https://docs.google.com/spreadsheets/d/14m-Sk89NxBeh9pnqrtYr-Vdi92INzIFF1uxO7lpTDuk/edit#gid=1175515436"
# copied
url_file="https://docs.google.com/spreadsheets/d/1CgHfrscwLZlU7gjk1IBGARlxEEsFML9YMiuEsaA9FWE/edit#gid=994132924"
worksheet_name="T6"
company="ACO"

In [68]:
await ggSheet.set_worksheet(url_file, worksheet_name)
row_date_header = await ggSheet.get_row_date_header(row_of_date)
all_records = await ggSheet.get_all_row_values()

In [69]:
handle_timesheet_sheet()

idx: 5
Project code: HR01_ACO.00
idx: 6
Project code: HR01_ACO.00
Employee name: Thiện Lê Phú
employee: {'name': 'HR-EMP-00043'}
Employee code: HR-EMP-00043
Timesheet doc_name existed {'name': 'TS-23-05-24-06074_HR-EMP-00043'}
{'doctype': 'Timesheet', 'name': 'TS-23-05-24-06074_HR-EMP-00043', 'employee': 'HR-EMP-00043', 'customer': 'ACO', 'parent_project': 'HR01_ACO.00', 'custom_task': '', 'custom_phase': 'Other', 'time_logs': [{'activity_type': '4001', 'from_time': '2024-04-18', 'hours': 8.0, 'project': 'HR01_ACO.00', 'task': 'bia0dlhuhn'}, {'activity_type': '4001', 'from_time': '2024-04-27', 'hours': 4.0, 'project': 'HR01_ACO.00', 'task': 'bia0dlhuhn'}, {'activity_type': '4001', 'from_time': '2024-04-29', 'hours': 8.0, 'project': 'HR01_ACO.00', 'task': 'bia0dlhuhn'}, {'activity_type': '4001', 'from_time': '2024-04-30', 'hours': 8.0, 'project': 'HR01_ACO.00', 'task': 'bia0dlhuhn'}, {'activity_type': '4001', 'from_time': '2024-05-01', 'hours': 8.0, 'project': 'HR01_ACO.00', 'task': 'bi

## TEAM CIVIL

In [70]:
# team_civil
# url_file="https://docs.google.com/spreadsheets/d/1qiVQptv2NK08Nr44ZapI8QTEHoDf1DhSpN0jkEe9_Ks/edit#gid=903792263"
# copied
url_file=""
worksheet_name="CIVIL"
company="ACO"

In [ ]:
await ggSheet.set_worksheet(url_file, worksheet_name)
row_date_header = await ggSheet.get_row_date_header(row_of_date)
all_records = await ggSheet.get_all_row_values()

In [ ]:
handle_timesheet_sheet()

## TEAM RND

In [71]:
# team_rnd
# url_file="https://docs.google.com/spreadsheets/d/15JPQLVqA_WDLmK00AsCnYrv1xqGzWwV9xbung_JDBmU/edit#gid=1885142845"
# copied
url_file="https://docs.google.com/spreadsheets/d/1EhlTU0LKzv-OsngOaZ9rXpzOPXmAureMjHrRo2OecTw/edit#gid=1885142845"
worksheet_name="RnD"
company="ACO"

In [72]:
await ggSheet.set_worksheet(url_file, worksheet_name)
row_date_header = await ggSheet.get_row_date_header(row_of_date)
all_records = await ggSheet.get_all_row_values()

In [73]:
handle_timesheet_sheet()

idx: 5
Project code: P000_ACO.00
idx: 6
Project code: P000_ACO.00
Employee name: Việt Nguyễn Hồng
employee: {'name': 'HR-EMP-00040'}
Employee code: HR-EMP-00040
Timesheet doc_name existed {'name': 'TS-23-05-24-06162_HR-EMP-00040'}
{'doctype': 'Timesheet', 'name': 'TS-23-05-24-06162_HR-EMP-00040', 'employee': 'HR-EMP-00040', 'customer': 'ACO', 'parent_project': 'P000_ACO.00', 'custom_task': 'Gernal Task', 'custom_phase': 'Other', 'time_logs': [{'activity_type': '2000', 'from_time': '2024-05-20', 'hours': 2.0, 'project': 'P000_ACO.00', 'task': 'diit0en9af'}]}
idx: 7
idx: 8
idx: 9
idx: 10
idx: 11
idx: 12
Project code: I00124_RSCI.00
idx: 13
Project code: I00124_RSCI.00
Employee name: Sokkheang Chan
employee: {'name': 'HR-EMP-00050'}
Employee code: HR-EMP-00050
Timesheet doc_name existed {'name': 'TS-23-05-24-06163_HR-EMP-00050'}
{'doctype': 'Timesheet', 'name': 'TS-23-05-24-06163_HR-EMP-00050', 'employee': 'HR-EMP-00050', 'customer': 'ACO', 'parent_project': 'I00124_RSCI.00', 'custom_task

In [ ]:







# team_bap
# url_file="https://docs.google.com/spreadsheets/d/11y58PE9RM16JozFR8yT-oV3K4etIsp2oFmiCsYVaydk/edit#gid=1885142845"
# copied
url_file="https://docs.google.com/spreadsheets/d/12bDZbCnJVkFTJTeMBmgsrAEejVKQCXa11M951ylOQVw/edit#gid=1885142845"
worksheet_name="BAP"
company="BAP"

